# <font color=green>1. Projeto Web Scraping

# 1.1. Introdução

## O objetivo deste projeto é coletar dados de um site (*Web Scraping*), de forma automática, utilizando comandos e bibliotecas em Python.

# 1.2. Ambiente e bibliotecas
### Utilizaremos em nosso treinamento o navegador Google Chrome

In [1]:
import bs4
import urllib.request as urllib_request
from urllib.request import urlopen, urlretrieve
import pandas
import math

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pandas.__version__)

BeautifulSoup -> 4.9.3
urllib -> 3.8
pandas -> 1.1.3


# 1.3. Restrições

## Para reduzir o tempo de processamento, filtramos no site somente veículos anunciados na Zona Sul da cidade do Rio de Janeiro. Iremos armazenar somente alguns atributos dos veículos pelo mesmo motivo. Porém o processo para obter todas as informações é o mesmo.

---
# <font color=green>2. Código

In [132]:
# Importando Bibliotecas
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd
import math

# Declarando variável cards
cards = []

## Obtendo o HTML
## A url utilizada é a obtida após a aplicação dos filtros no site da OLX

req = Request('https://rj.olx.com.br/rio-de-janeiro-e-regiao/zona-sul/autos-e-pecas/carros-vans-e-utilitarios',headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = webpage.decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

#Obtendo o número total de páginas para iterar no site
pages = int(soup.find('span', class_="sc-1mi5vq6-0 eDXljX sc-ifAKCX fhJlIo").get_text().split()[-2].split('.')[0]+soup.find('span', class_="sc-1mi5vq6-0 eDXljX sc-ifAKCX fhJlIo").get_text().split()[-2].split('.')[1])/int(soup.find('span', class_="sc-1mi5vq6-0 eDXljX sc-ifAKCX fhJlIo").get_text().split()[2])
pages = math.ceil(pages)

## Iterando por todas as páginas do site
for i in range(pages):
    ## Obtendo o HTML
    req = Request('https://rj.olx.com.br/rio-de-janeiro-e-regiao/zona-sul/autos-e-pecas/carros-vans-e-utilitarios?o='+str(i+1),headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    html = webpage.decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    # Obtendo as TAGs de interesse
    anuncios = soup.find('ul', {"class": "sc-1fcmfeb-1 kntIvV"}).findAll('div', {"class": "fnmrjs-4 jflPgo"})

    # Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Nome
        card['nome'] = anuncio.find('h2').get_text()
        
        # Valor
        card['valor'] = anuncio.find('div', {"class": "aoie8y-0"}).find('span').get_text()
        
        #Adicionando resultado a lista cards
        cards.append(card)
        
# Criando um DataFrame com os resultados
dataset = pd.DataFrame(cards)
dataset.to_csv('dataset2.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset

,nome,valor
0,Fiesta 2010 1.6 raridade.,R$ 19.500
1,Punto Essence 1.6 26000 km impecavel,R$ 46.890
2,HB 20 S 1.6 comfort,R$ 56.890
3,VENDO VAN RENAULT MASTER,
4,Gol G5 2013 Completo 59mil KM+Novo Do Brasil+T...,R$ 25.900
...,...,...
1393,Bmw x1 2.0 16v Turbo Activeflex Sdrive20i,R$ 82.500
1394,Kia Sportage 2.0 ex 4x2 16v,R$ 109.500
1395,Bmw 320i 2.0 16v Turbo m Sport,R$ 209.500
1396,Honda Civic 2.0 16vone Sport,R$ 84.500
